In [1]:
import pandas as pd
import re, time, requests, sys, json, tqdm
from selenium import webdriver
from bs4 import BeautifulSoup
from ipywidgets import IntProgress
from IPython.display import display

In [2]:
def LoginFacebook(email, pwd):
    #前往臉書
    driver.get("http://www.facebook.com")
    
    #搜尋和輸入帳號密碼
    Email_Input = driver.find_element_by_id("email")
    Password_Input  = driver.find_element_by_id("pass")
    
    try:
        SubmitBtn   = driver.find_element_by_id("loginbutton")
    except:
        SubmitBtn   = driver.find_element_by_id("login")
        
    #輸入帳密   
    Email_Input.send_keys(email)
    Password_Input.send_keys(pwd)
    
    #按下登入按鈕
    SubmitBtn.click()

In [3]:
# 取得連結
# def FindLinks(url, n):
#     Links = []
#     driver.get(url)
    
#     for i in range(n):
#         #下拉至頁面底端
#         driver.execute_script('window.scrollTo(0, document.body.scrollHeight);')
#         #等待 3 秒 
#         time.sleep(3)
        
#     #尋找
#     driver.find_element_by_xpath('//a[@id="expanding_cta_close_button"]').click()
#     soup = BeautifulSoup(driver.page_source)
#     posts = soup.findAll('div', {'class':'clearfix y_c3pyo2ta3'})
    
#     for i in posts:
#         Links.append('https://www.facebook.com' + i.find('a',{'class':'_5pcq'}).attrs['href'].split('?',2)[0])
        
#     return Links

In [4]:
#內文，基本資料
def PostContent(soup,link):
    #粉專名稱
    try:
        Name = soup.find('h3', class_='_52jd _52jb _52jh _5qc3 _4vc- _3rc4 _4vc-').text
    except:
        try:
            Name = soup.find('div', class_='msg').find('strong', class_='actor').text
        except:
            try:
                Name = soup.find('h3', class_='_52jd _52jb _52jg _5qc3 _4vc- _3rc4 _4vc-').find('strong').text
            except:
                Name = 'None'
        
    #超連結
    try:
        Link = link
    except:
        Link = 'None'
        
    #發文時間    
    try:
        Time = soup.find('div', class_='_52jc _5qc4 _78cz _24u0 _36xo').text
    except:
        try:
            Time = soup.find('div', class_='_2vja mfss fcg').find('abbr').text
        except:
            Time = 'None'
        
    # 文章內容
    try:
        Content = soup.find('div', class_='_5rgt _5nk5').text
    except:
        try:
            Content = soup.find('div', class_='msg').find('div', class_='').text
        except:
            Content = 'None'
        
    print('Name:{}\nLink:{}\nTime:{}\nContent:{}'.format(Name, Link, Time, Content))
    
    #寫入寫入DataFrame
    Post_DataFrame = pd.DataFrame(data = [
            {
                'Name':Name,
                'Link' : Link,
                'Time' : Time,
                'Content' : Content
            }
        ],columns = [
            'Name', 
            'Link', 
            'Time', 
            'Content'
        ])
    
    return Post_DataFrame

In [5]:
# 留言
def CrawlComment(soup, link):
    #建立DataFrame
    Comment_DataFrame = pd.DataFrame()
    
    #顯示進度條
    comment_progress = IntProgress()
    display(comment_progress)
    
    #取得貼文連結
    Post_Link = link
    
    #爬取前30篇     
    Top_Thirty_Comment_List = soup.findAll('div', class_='_2a_i')
    for i in Top_Thirty_Comment_List:
        User_ID = i.find('div', class_='_2a_j')['data-sigil'].split('feed_story_ring')[1]
        Comment = i.find('div', {'data-sigil':'comment-body'}).text
        if i.find('div', {'data-sigil':'comment-body'}).has_attr('class'):    
            Comment = 'Sticker'
        Comment_Time = i.find('abbr', class_="_4ghv _2b0a").text
        #判斷是否為頭號粉絲     
        try:
            Fan_Type = i.find('div', class_='_2b05').find('div', class_='_7_cb _3-8m').text
            Fan_Type = 1
        except:
            Fan_Type = 0
        #判斷是有附圖片、影片
        try:
            i.find('div', class_='_14v5').find('div', class_='_2b1t attachment').text
            Attachment = 1
        except: 
            Attachment = 0
        
        #寫入DataFrame    
        Comment_DataFrame = WriteCommentToDF(Comment_DataFrame, Post_Link, User_ID, Comment, Comment_Time, Attachment, Fan_Type)
        
    # 前往API
    API_url = soup.find("a", {"class":"_108_"})['data-ajaxify-href']
    API_url = 'https://mobile.facebook.com{}'.format(API_url)
    driver.get(API_url)
    
    
    #迴圈爬取全部留言
    loop = True
    while loop:
        # 編碼為unicode-escape and html 格式
        soup = BeautifulSoup(driver.page_source)
        Text = soup.text
        Decode_Text = Text.encode().decode('unicode-escape').replace('\\','')

        # 取得評論清單
        Comment_List = Decode_Text.split('_2a_i')[1:]
        
        # 取出評論與USER_ID
        for i in range(len(Comment_List)):
            
            #搜尋ID
            User_ID = Comment_List[i].split('<a class="_14v8')[0].split('data-sigil="feed_story_ring')[1].split('">')[0]
            #評論#處理表情符號
            Comment = str(BeautifulSoup(Comment_List[i].split('comment-body">')[1].split('<a class="_14v8')[0]).text)
            Comment = Comment.encode('utf-16', 'surrogatepass').decode('utf-16')
            #評論時間
            Comment_Time = Comment_List[i].split('_4ghv _2b0a">')[1].split('</abbr>')[0]
            Attachment = 1
            Fan_Type = 0
            
            #判別          
            try:
                #判別是否有附件    
                if '_2b1t attachment' not in Comment_List[i]:
                    Attachment = 0
                #判別是否為頭號粉絲
                if '_7_cb _3-8m' in Comment_List[i].split('comment-body">')[1].split('<a class="_14v8')[0]:
                    Fan_Type = 1
                #判別評論是否只有圖片
                if '_4kk6' in Comment_List[i].split('comment-body">')[1].split('<a class="_14v8')[0]:
                    Comment = 'Sticker'
            except:
                print('error')
                
            #寫入DataFrame    
            Comment_DataFrame = WriteCommentToDF(Comment_DataFrame, Post_Link, User_ID, Comment, Comment_Time, Attachment, Fan_Type)
            
        
        #取得下一個API 的 Ul
        try:
            Head_index = Text.find('_108_')
            Tail_index = Text.find('_5msp')
            Next_API_url = 'https://m.facebook.com/ajax{}'.format(Text[Head_index:Tail_index].split()[1][27:].split('\\')[0])
           
            Total = Next_API_url.split('count=')[1].split('&')[0]
            Num = Next_API_url.split('p=')[1].split('&')[0]
            
            comment_progress.max = float(Total)
            comment_progress.value = float(Num)
            comment_progress.description = str(((float(Num))/float(Total))*100)
            #前往 下30筆留言API超連結
            driver.get(Next_API_url)
        except:
            loop = False
            comment_progress.description = '完成'
            pass
        
    return Comment_DataFrame

def WriteCommentToDF(Comment_DataFrame, Post_Link, User_ID, Comment, Comment_Time, Attachment, Fan_Type):
    #寫入DataFrame
    Comment_Result = pd.DataFrame(data = [
        {
            'Post_Link' : Post_Link,
            'User_ID' : User_ID,
            'Comment' : Comment,
            'Comment_Time' : Comment_Time,
            'Attachment' : Attachment,
            'Fan_Type' : Fan_Type
        }
    ],columns = [
        'Post_Link', 
        'User_ID', 
        'Comment',
        'Comment_Time',
        'Attachment',
        'Fan_Type'
    ])
    
    return pd.concat([Comment_DataFrame, Comment_Result], ignore_index=True)

# main 

In [7]:
# 開啟瀏覽器
driver = webdriver.Chrome(executable_path='./cdriver')

# 帳號 密碼
email = 'your email'
pwd = 'your password'
# 登入
LoginFacebook(email, pwd)

In [10]:
# Main

# 計時花費時間
start_time = time.time()
post_index = 0



# 取得貼文連結
LinkFile = pd.read_excel('your link file')
Links = LinkFile['url'].tolist()

# 建立DF
PostsInformation = pd.DataFrame()
PostsComments = pd.DataFrame()

for link in Links:
    print(link,' 處理中...')
    try:
        driver.get(link)
        #如果是 video 頁面，要再跳轉一次
        if bool(re.search("watch",driver.current_url)):
            driver.get(driver.find_element_by_xpath('//div[@class="_34qc _3hxn _3myz _4b45"]//a').get_attribute('href'))
        
        soup = BeautifulSoup(driver.page_source)
        PostsInformation = pd.concat([PostsInformation, PostContent(soup,link)],ignore_index=True)
        PostsComments = pd.concat([PostsComments, CrawlComment(soup,link)],ignore_index=True)
    except:
        print('Error: ', sys.exc_info())
print("--- %s 秒完成 ---" % (time.time() - start_time))

https://m.facebook.com/tsaiingwen/posts/10156349477186065  處理中...
Name:蔡英文 Tsai Ing-wen
Link:https://m.facebook.com/tsaiingwen/posts/10156349477186065
Time:January 5 at 4:45 PM · 
Content:我在台南的造勢活動現場，也在這裡的朋友留言讓我知道好嗎？ 立委 第一選區3號賴惠員 第二選區3號郭國文 第三選區3號陳亭妃 第四選區6號林宜瑾 第五選區3號林俊憲 第六選區4號王定宇


IntProgress(value=0)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:49: DeprecationWarning: invalid escape sequence '\/'


https://m.facebook.com/tsaiingwen/photos/a.390960786064/10156349665556065/  處理中...
Name:蔡英文 Tsai Ing-wen
Link:https://m.facebook.com/tsaiingwen/photos/a.390960786064/10156349665556065/
Time:Jan 5
Content:當國人為國軍將士的殉職全民一體、同感哀傷的時刻，韓國瑜市長卻用「台灣中邪了、國運不昌隆」這樣的話語來消費將士的犧牲，相信全體國人都無法接受這樣的話語，而同感憤慨。 　　 我要正告韓國瑜市長和國民黨陣營：不應該一再消費台灣人民的傷痛，作為選舉抹黑的題材，那對逝者是最大的不敬，也是對生者最殘忍的傷害。 　　 台灣的國運，一直都繫於2300萬全體國人的共同努力、團結奮起。我也要說，台灣的未來，絕不能有一個失格的領導者，會用輕蔑失格的語言，來消費踐踏人民的傷痛，造成台灣社會的紛擾不止、國家的前景堪憂。韓市長離譜的言行，應該深切反省，向家屬、國軍與國人道歉。


IntProgress(value=0)

...

https://m.facebook.com/tsaiingwen/photos/a.390960786064/10156344742196065/  處理中...
Name:蔡英文 Tsai Ing-wen
Link:https://m.facebook.com/tsaiingwen/photos/a.390960786064/10156344742196065/
Time:Jan 3
Content:穩定軍心！安定民心！ 　　 今天我到國防部與衡山指揮所視察，為我們國軍弟兄姊妺們打氣。 　　 我知道，參謀總長和幾位同仁的殉職，讓大家都感到不捨。在這個時候，我希望我們國軍更要堅守崗位，扛起國家戰備的任務，讓國防事務如常運作，讓國人安心。 　　 我們一起加油！


IntProgress(value=0)

https://m.facebook.com/tsaiingwen/photos/a.390960786064/10156344267611065/  處理中...
Name:蔡英文 Tsai Ing-wen
Link:https://m.facebook.com/tsaiingwen/photos/a.390960786064/10156344267611065/
Time:Jan 3
Content:今天早上，我召開「國防軍事會談」，除了為殉職弟兄默哀，也下達三項任務，要求國軍重要幹部，務必確保國防事務如常運作。繼續堅守崗位，就是紀念沈總長的最好方式。 　　 第一個任務，務必確保軍心民心穩定：國防事務有完整分工，也有代理制度，國軍要堅守崗位，讓國防事務如常運作。 　　 第二個任務，務必確保台海周邊安全：國軍要密切注意台海軍事動態，第一時間快速反應、部署，確保國家安全。 　　 第三個任務，務必確保裝備檢整完善：國軍幹部要嚴格要求各部隊，詳細檢整裝備，維持妥善率，不能有任何鬆懈。 　　 讓軍心穩定，民心如常，是三軍統帥及國軍將領們的責任。這「三個確保」，我們會嚴格督導、認真執行，確保台灣的安全。


IntProgress(value=0)

--- 1626.227699995041 秒完成 ---


In [11]:
PostsInformation

,Name,Link,Time,Content
0,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/posts/101563...,January 5 at 4:45 PM ·,我在台南的造勢活動現場，也在這裡的朋友留言讓我知道好嗎？ 立委 第一選區3號賴惠員 第二選區...
1,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.390...,Jan 5,當國人為國軍將士的殉職全民一體、同感哀傷的時刻，韓國瑜市長卻用「台灣中邪了、國運不昌隆」這樣...
2,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.390...,Jan 5,今天下午15:12分，TVBS將播出這段時間採訪我和我的幕僚的專題「蔡英文改變進行式」，大家...
3,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.390...,Jan 5,你收到選舉公報了嗎？距離投票的日子只剩下6天，準備好了吧！ 如果你會去投票，請留言 #...
4,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/videos/26558...,January 5 at 9:28 AM ·,全台大掃街，從我的家鄉屏東出發！ 距離投票日剩下最後6天，明天開始我會跟 賴清德 副總...
5,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/posts/101563...,January 4 at 6:05 PM ·,各位弟兄姊妹，我代替沈總長來看大家了 今天我到空軍第一雷達分隊視察，替沈總長完成他原先...
6,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.390...,Jan 4,這兩天，競辦發言人的專訪內容，因為錯誤的表達引起誤會。我想藉這個機會，重申我一直以來明確的態...
7,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.101...,Jan 3,面對生命的無常，我們有很多的不捨與煎熬。今天晚上，我們舉行了追思活動，讓我們做彼此的肩膀，為...
8,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.390...,Jan 3,穩定軍心！安定民心！ 今天我到國防部與衡山指揮所視察，為我們國軍弟兄姊妺們打氣。 ...
9,蔡英文 Tsai Ing-wen,https://m.facebook.com/tsaiingwen/photos/a.390...,Jan 3,今天早上，我召開「國防軍事會談」，除了為殉職弟兄默哀，也下達三項任務，要求國軍重要幹部，務必...


In [12]:
PostsComments

,Post_Link,User_ID,Comment,Comment_Time,Attachment,Fan_Type
0,https://m.facebook.com/tsaiingwen/posts/101563...,100007949447494,唯一不變的堅持！民進黨加油 全壘打👍,2 mos,0,0
1,https://m.facebook.com/tsaiingwen/posts/101563...,100005895276120,我愛台灣，所以我也在現場，大家一起加油💪,2 mos,0,0
2,https://m.facebook.com/tsaiingwen/posts/101563...,691610069,台灣人一定一定要投票 要不然下次走上街頭是你們的了 我們香港人為你們加油,2 mos,0,0
3,https://m.facebook.com/tsaiingwen/posts/101563...,100000510090981,2020台灣要贏！謝謝小英妳辛苦了！加油💪,2 mos,0,0
4,https://m.facebook.com/tsaiingwen/posts/101563...,100004713856532,台灣加油，下架吳斯懷,2 mos,0,0
...,...,...,...,...,...,...
28221,https://m.facebook.com/tsaiingwen/photos/a.390...,100031392452407,一堆人留言真他媽的狗兒子一樣！說什麼發生了才知道檢討？是喝神仙水嗎能預判國家大事嗎？狗兒子都...,2 mos,0,0
28222,https://m.facebook.com/tsaiingwen/photos/a.390...,100003686517730,Sticker,2 mos,0,0
28223,https://m.facebook.com/tsaiingwen/photos/a.390...,100000294513304,他就是一隻非洲豬溫早死早下地獄可悪,2 mos,0,0
28224,https://m.facebook.com/tsaiingwen/photos/a.390...,100040483580756,蔡英文總統加油💪 台灣加油💪 #台灣人團結 #致敬救援人員及國軍弟兄,2 mos,0,0


In [13]:
# 寫為csv檔案
PostsInformation.to_csv('Posts_Infomation{}.csv'.format(int(time.time())), encoding='utf_8_sig')
PostsComments.to_csv('Posts_Comments{}.csv'.format(int(time.time())), encoding='utf_8_sig')